# 6.864 Project
$\textbf{Main idea}$: Testing the limits of zero-shot and few-shot learning for labeling at scale.

Techniques to try (unsupervised)
1. Baseline: GloVe embedding-based similarity scoring [Elaine]
2. SBERT: SBERT based similarity scoring [Brian]
Encode label embedding. Encode query embedding. Get dot product of query vs. label 1,... 30, predict highest dot product.  
3. Feed SBERT top dot products into ZSL. [Brian]

Techniques to try (semi-supervised)
1. SBERT nearest neighbors (65%) [Brian]
2. Siamese neural nets [Brian]
3. Both can also contribute to ZSL [Elaine]

Technique to try (supervised)
1. BERT Supervised classification [Elaine] 

Final report main table
# training examples | Model structure | Accuracy 
0 | GloVe (baseline) | 30% 

Q = " I returned something to a  store but can't see my refund."
--> SBERT NN --> hypothesis_set = ['refund', 'lost card', 'chargeback']
--> ZSL(Q, hypothesis_set) --> [0.8, 0.2, 0.0] 

SBERT: sentence --> 768 dimensional vector
label 1, ... 30  
sentence embeddings : get label centroids by taking the mean of all sentence embeddings corresponding to label 1,...30. 1000 labels 

New sentence: --> SBERT --> get nearest centroid --> predict label: ~65% 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%%bash
# Logistics #2: install the transformers package, create a folder, download the dataset and a patch
pip -q install transformers
pip -q install datasets
pip -q install tqdm
pip -q install sentencepiece 

# mkdir "/content/gdrive/MyDrive/6864_Project/"
cd "/content/gdrive/MyDrive/6864_Project/"

In [ ]:
import numpy as np
import pandas as pd
import re
import random

import os
os.chdir('/content/gdrive/MyDrive/6864_Project/')

In [ ]:
from datasets import load_dataset

banking = load_dataset('banking77')
print(banking)

Using custom data configuration default


Dataset banking77 downloaded and prepared to /root/.cache/huggingface/datasets/banking77/default/1.1.0/17ffc2ed47c2ed928bee64127ff1dbc97204cb974c2f980becae7c864007aed9. Subsequent calls will reuse this data.
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10003
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3080
    })
})


In [ ]:
# Need to remap all labels
label_tbl = pd.read_csv('banking77.csv')
label_map = {}

# for simplicity only keep some 30 labels in training/testing data
random.seed(6864)
n_classes = 30
labels_to_keep = random.sample(list(range(len(label_tbl))), k= n_classes)

for i in range(len(label_tbl)):
    label = label_tbl['Label'].iloc[i]
    label = re.sub(r'\_',' ',label)
    ix = label_tbl['Label_ix'].iloc[i]
    if ix in labels_to_keep:
      label_map.update({ix: label})

In [ ]:
# filter trianing set to only intents we want 
banking2 = banking.filter(lambda row: row['label'] in labels_to_keep)

print(len(banking2['train']))
print(len(banking2['test']))


4001
1200


In [ ]:
i=1012
print("input:", banking2['train'][i]['text'])
print("label:", label_map[banking2['train'][i]['label']])
print()
i=100
print("input:", banking2['train'][i]['text'])
print("label:", label_map[banking2['train'][i]['label']])
print()
i=2000
print("input:", banking2['train'][i]['text'])
print("label:", label_map[banking2['train'][i]['label']])

input: I tried to top up, but it didn't finish.
label: pending top up

input: can you share card tracking number?
label: card arrival

input: I returned something to a  store but can't see my refund.
label: refund not showing up


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

"""
The following is from the sentence bert (SBERT) framework
https://huggingface.co/sentence-transformers/bert-base-nli-mean-tokens
"""

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

#Sentences we want sentence embeddings for
query = ['I returned something to a  store but can\'t see my refund.'] # TRUE INTENT: 'refund not showing up'
intent_set = list(label_map.values())

#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens")
lm = AutoModel.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens").cuda()

#Tokenize sentences and place them on tensor
encoded_query = tokenizer(query, padding=True, truncation=True,return_token_type_ids=False, max_length=128, return_tensors='pt')
encoded_query['input_ids'] = encoded_query['input_ids'].cuda()
encoded_query['attention_mask'] = encoded_query['attention_mask'].cuda()
encoded_intents = tokenizer(intent_set, padding=True, truncation=True,return_token_type_ids=False, max_length=128, return_tensors='pt')
encoded_intents['input_ids'] = encoded_intents['input_ids'].cuda()
encoded_intents['attention_mask'] = encoded_intents['attention_mask'].cuda()


#Compute token embeddings
lm.eval()
with torch.no_grad():
    output_query = lm(**encoded_query)
    output_intents = lm(**encoded_intents)

#Perform pooling. In this case, mean pooling
query_embeddings = mean_pooling(output_query, encoded_query['attention_mask'])
intents_embeddings_raw = mean_pooling(output_intents, encoded_intents['attention_mask'])


In [ ]:
# Now take the dot product to find out which ones are closest
# We see that the true intent is the third one. But considering this is a baseline model this isn't unexpected 
top_intents = torch.topk(torch.inner(query_embeddings, intents_embeddings_raw), k = 5).indices.squeeze()
for i in range(len(top_intents)):
  print(i+1,". ",intent_set[top_intents[i].item()])

1 .  balance not updated after bank transfer
2 .  balance not updated after cheque or cash deposit
3 .  refund not showing up
4 .  declined card payment
5 .  lost or stolen phone


# Siamese neural network implementation
The basic idea is that we train a siamese neural network based on the triplet loss function for few-shot learning. In the triplet loss framework we provide a query that's of the same class and another query that is not of the same class.

After training the model to distinguish which query is of the same class and which is not, we can use the network for unseen queries. We can then feed in the new query along with all of the label centroids to determine which label centroid is closest to the new query. 

In [ ]:
# First create a relatively small training set in light of few-shot learning. We collect k samples from each label
label_counter = {}
n_samples = 20 # This designates the number of each sentence with label i that we have in training
train_queries, train_labels = [],[]
test_queries, test_labels = [], []
for i in range(len(banking2['train'])):
  text = banking2['train'][i]['text']
  label = banking2['train'][i]['label']
  if label not in label_counter:
    label_counter.update({label: 1})
    train_queries.append(text)
    train_labels.append(label)
  else:
    if label_counter[label] <= n_samples:
      train_queries.append(text)
      train_labels.append(label)
      label_counter[label] += 1
    else:
      test_queries.append(text)
      test_labels.append(label)

print(len(train_queries))
print(len(test_queries))

630
3371


In [ ]:
# Next embed the training queries and create centroids based on the train

encoded_query = tokenizer(train_queries,return_token_type_ids=False, padding=True,  max_length=128, return_tensors='pt')
train_input_ids = encoded_query['input_ids'].cuda()
train_attn_mask = encoded_query['attention_mask'].cuda()
with torch.no_grad():
    output_query = lm(input_ids = train_input_ids, 
                         attention_mask = train_attn_mask) #unpack arguments 

query_embeddings = mean_pooling(output_query, train_attn_mask)

print(query_embeddings.shape)

torch.Size([630, 768])


In [ ]:
intent_centroids = {}
intent_centroids_lm = torch.zeros(intents_embeddings.shape).cuda()

# train_labels = torch.tensor(train_labels)
# test_labels = torch.tensor(test_labels)

# centroids = torch.zeros((len(set(train_labels)), query_embeddings.shape[1]))
i=0
for label in set(train_labels):
  ix = np.where(np.array(train_labels) == label)[0]
  intent_centroids[label] = torch.mean(query_embeddings[ix,:], dim = 0)
  intent_centroids_lm[i,:] = torch.mean(query_embeddings[ix,:], dim = 0)
  i+=1

print("Created {} centroids based on query embeddings and label groups.".format(n_classes))
print(intent_centroids.keys())

Created 30 centroids based on query embeddings and label groups.
dict_keys([0, 1, 5, 6, 11, 17, 19, 23, 25, 31, 32, 34, 36, 37, 39, 42, 43, 47, 49, 51, 52, 55, 58, 59, 61, 62, 63, 64, 69, 72])


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class SiameseNet(nn.Module):
    def __init__(self, lm):
      super(SiameseNet, self).__init__()
      torch.manual_seed(6864)
      self.lm = lm
      self.embedding_dim = 768
      self.output_size = 64

      self.lin = nn.Linear(self.embedding_dim, 256)
      self.lin1 = nn.Linear(256, 128)
      self.out = nn.Linear(128, self.output_size)
      self.dropout = nn.Dropout(0.5)

    def freeze_lm(self): # freeze the SBERT model such that we only train the classifier 
      for param in self.lm.parameters():
        param.requires_grad = False

    def FF(self, embeddings): # Feed forward embedding 
      x = F.relu(self.lin(embeddings))
      x = self.dropout(x)
      x = F.relu(self.lin1(x))
      x = self.out(x)
      return x

    def forward_one(self, encoded_query): # define the same forward function for the embeddings to produce embedding representation
      output = self.lm(**encoded_query)
      embeddings = mean_pooling(output, encoded_query['attention_mask']) # pool -> (batch size, hidden_size)
      x = self.FF(embeddings)
      return x

    def forward(self, anc_encoded, pos_encoded, neg_encoded): # takes in anchor, positive, and negative
      anc_emb = self.forward_one(anc_encoded) # pool -> (batch size, 32)
      pos_emb = self.forward_one(pos_encoded)
      neg_emb = self.forward_one(neg_encoded)
      return anc_emb, pos_emb, neg_emb


# define a loss function
from torch.nn.modules.distance import PairwiseDistance
class TripletLoss(nn.Module):
    def __init__(self, eps): # eps is some margin 
        super(TripletLoss, self).__init__()
        self.eps = eps
        self.dist_fcn = PairwiseDistance(p=2)

    def forward(self, anc_emb, pos_emb, neg_emb):
        pos_dist = self.dist_fcn(anc_emb, pos_emb)
        neg_dist = self.dist_fcn(anc_emb, neg_emb)

        hinge_dist = torch.clamp(self.eps + pos_dist - neg_dist, min=0.0)
        loss = torch.mean(hinge_dist) # take the average distance in the batch to be the loss  
        return loss

In [ ]:
# # DEMO OF PROPOGATING THROUGH TO TRAINING LOSS 


# batch_ix = np.array([])
# np.random.seed(1)
# label_set = list(set(train_labels))
# for label in label_set:
#   label_ix = np.where(np.array(label)==np.array(train_labels))[0]
#   label_ix = np.random.choice(label_ix,size=n_pos, replace = False) 
#   batch_ix = np.concatenate([batch_ix, label_ix]).astype(np.int32)
# batch_size = len(batch_ix)
# batch_anc_enc = {'input_ids': train_input_ids[batch_ix], 'attention_mask': train_attn_mask[batch_ix]}
# # Embed everything in the sample and obtain the distance matrix
# embeddings = Siamese.forward_one(batch_anc_enc)

# Triplet selection
Given $x_{i}^{a}$ we want to select positive anchor (within a batch) such that $x_{i}^{p} = argmax||f(x_{i}^{a})-f(x_{i}^{p})||_{2}^{2}$ (it looks far from anchor, but is actually the same class) and similarly we want $x_{i}^{n} = argmin||f(x_{i}^{a})-f(x_{i}^{n})||_{2}^{2}$ (it looks close together but is actually of a different class). 

We can also consider "semi-hard" examples which have the property that $||f(x_{i}^{a})-f(x_{i}^{p})||_{2}^{2} \leq ||f(x_{i}^{a})-f(x_{i}^{n})||_{2}^{2}$. 

Lastly, the Facenet paper recommends "Instead of picking the hardest positive, we use all anchor-positive pairs in a mini-batch while still selecting the hard negatives." In similar vein, we can include all anchor-centroid pairs so that all labels eventually learn that they belong with a certain centroid.

Ultimately, we have to make sure that we have some positive number of examples for each class. Because we are treating this as a semi-supervised problem, we're going to assume that we have limited labels (e.g. 20 labels of each class) and will instead use random sampling to create batches.

In [ ]:
# Wrap the batch generation into a function 
def generate_batch(model, train_input_ids, train_attn_mask, train_labels, centroids, n_pos, random_state):
  """
  Model: the Siamese net model that will be used to generate embeddings
  train_input_ids: ALL tokenized input ids across the training set
  train_attention mask: ALL tokenized attention masks across the training set
  train_labels: ALL labels across the training set
  centroids: A dictionary that maps {label: centroid embedding}  
  n_pos: The number of examples to draw from each class in train_labels
  random_state: The seet to be set 
  """

  batch_ix = np.array([])
  np.random.seed(random_state)
  label_set = list(set(train_labels))
  for label in label_set:
    label_ix = np.where(np.array(label)==np.array(train_labels))[0]
    label_ix = np.random.choice(label_ix,size=n_pos, replace = False) 
    batch_ix = np.concatenate([batch_ix, label_ix]).astype(np.int32)
  batch_size = len(batch_ix)
  batch_anc_enc = {'input_ids': train_input_ids[batch_ix], 'attention_mask': train_attn_mask[batch_ix]}
  # Embed everything in the sample and obtain the distance matrix
  embeddings = model.forward_one(batch_anc_enc)
  emb_dists = torch.cdist(embeddings,embeddings, p=2)
  # find the indices of emb_dists with the respective max (if same class) or min (if different class) for the distance
  pos_samples = torch.zeros((batch_size)).long()
  neg_samples = torch.zeros((batch_size)).long()
  # Get the embedding of the same label centroid as well as some embedding of a (random) different label centroid
  # Unlike pos/neg samples we'll just create this directly (not through input_ids and attention_mask)
  emb_shape = centroids[label_set[0]].shape[0] # 768 is the shape of each centroid 
  pos_centroid_samples = torch.zeros((batch_size,emb_shape))
  neg_centroid_samples = torch.zeros((batch_size,emb_shape))

  batch_labels = np.array(train_labels)[batch_ix]
  np.random.seed(random_state)
  for i in range(batch_size): # just do brute-force over all selected samples for each label - it's a small matrix, otherwise it's too difficult to deal with indices
    lab = batch_labels[i]
    pos_ix = np.where(lab == batch_labels)[0]
    pos_ix = np.setdiff1d(pos_ix, i) 
    neg_ix = np.where(lab != batch_labels)[0]
    pos_dists = emb_dists[i,pos_ix]
    neg_dists = emb_dists[i,neg_ix]
    pos_samples[i] = pos_ix[torch.argmax(pos_dists)] # Hard-positive
    neg_samples[i] = neg_ix[torch.argmin(neg_dists)] # Hard-negative
    pos_centroid_samples[i] = centroids[lab] # Centroid-positive
    diff_labels = [x for x in label_set if x != lab] # Randomly pick a different centroid to designate as a negative 
    neg_centroid_samples[i] = centroids[ np.random.choice(diff_labels, size=1, replace = False)[0] ] # Centroid-negative

  # now generate a batch-based set of (anchor, hard-positive, hard-negative) triplets
  pos_input_ids, pos_attention_mask = batch_anc_enc['input_ids'][pos_samples], batch_anc_enc['attention_mask'][pos_samples]
  neg_input_ids, neg_attention_mask = batch_anc_enc['input_ids'][neg_samples], batch_anc_enc['attention_mask'][neg_samples]

  batch_pos_enc = {'input_ids': pos_input_ids, 'attention_mask': pos_attention_mask}
  batch_neg_enc = {'input_ids': neg_input_ids, 'attention_mask': neg_attention_mask}

  return batch_anc_enc, batch_pos_enc, batch_neg_enc, pos_centroid_samples, neg_centroid_samples

In [ ]:
# Hyper-parameters:
import transformers

num_epochs = 5
num_batches = 10
learning_rate = 3e-3
weight_decay = 1e-5
optimizer_eps = 1e-6
triplet_eps = 1
n_pos = 5
warmup_rate = 0.05
ques_max_length = 64
max_grad_norm = 5
ctx_max_length = 448
batch_size = n_pos*n_classes*2 # (x2 because we have both hard p/n and centroid p/n)

# Calculating the number of warmup steps
num_training_cases = num_batches*n_pos*n_classes
t_total = (num_training_cases // batch_size + 1) * num_epochs
ext_warmup_steps = int(warmup_rate * t_total)

# Create model 
Siamese = SiameseNet(lm).cuda()
Siamese.freeze_lm()
loss_fn = TripletLoss(eps = triplet_eps)

# Initializing an AdamW optimizer
ext_optim = torch.optim.AdamW(Siamese.parameters(), lr=learning_rate,
                              eps=optimizer_eps, weight_decay=weight_decay)

# Initializing the learning rate scheduler [details are in the BERT paper]
ext_sche = transformers.get_linear_schedule_with_warmup(
    ext_optim, num_warmup_steps=ext_warmup_steps, num_training_steps=t_total
)

In [ ]:
# Define the training loop
num_epochs = 10
num_batches = 21
# Important to make sure all inputs (except train_labels) is on the same device 
Siamese.train()

torch.manual_seed(6864)
for epoch in range(num_epochs):
  for batch in range(num_batches):

    # Zero out gradient
    Siamese.zero_grad() 

    # Generate batch
    batch_anc_enc, batch_pos_enc, batch_neg_enc, pos_centroid, neg_centroid = generate_batch(Siamese, train_input_ids, train_attn_mask, train_labels, 
                                                                                             intent_centroids, n_pos, batch*epoch)
    # Encode anc/pos/neg
    anc_emb, pos_emb, neg_emb = Siamese(batch_anc_enc, batch_pos_enc, batch_neg_enc)

    # Embed centroids
    pos_centroid_emb = Siamese.FF(pos_centroid.cuda())
    neg_centroid_emb = Siamese.FF(neg_centroid.cuda())

    # compute the loss 
    main_loss = loss_fn(anc_emb, pos_emb, neg_emb)
    # print("Hard Pos/Neg Loss: {}".format(main_loss))

    centroid_loss = loss_fn(anc_emb, pos_centroid_emb, neg_centroid_emb)
    # print("Centroid Pos/Neg Loss: {}".format(centroid_loss))

    total_loss = main_loss + centroid_loss # concatenate the loss 
    if batch % 10 == 0:
      print("Epoch {} batch {} total loss: {}".format(epoch, batch, total_loss))

    total_loss.backward()
    # torch.nn.utils.clip_grad_norm_(Siamese.parameters(), max_grad_norm)

    ext_optim.step()
    ext_sche.step() # Update learning rate for better convergence

Epoch 0 batch 0 total loss: 1.9832000732421875
Epoch 0 batch 10 total loss: 1.2397146224975586
Epoch 0 batch 20 total loss: 1.0428435802459717
Epoch 1 batch 0 total loss: 0.8254457116127014
Epoch 1 batch 10 total loss: 1.4212294816970825
Epoch 1 batch 20 total loss: 1.2812693119049072
Epoch 2 batch 0 total loss: 0.8891004323959351
Epoch 2 batch 10 total loss: 1.4185035228729248
Epoch 2 batch 20 total loss: 1.4887597560882568
Epoch 3 batch 0 total loss: 0.864205002784729
Epoch 3 batch 10 total loss: 1.5550286769866943
Epoch 3 batch 20 total loss: 1.2407758235931396
Epoch 4 batch 0 total loss: 0.8707939386367798
Epoch 4 batch 10 total loss: 1.4016286134719849
Epoch 4 batch 20 total loss: 1.2705329656600952
Epoch 5 batch 0 total loss: 0.924686849117279
Epoch 5 batch 10 total loss: 1.5595921277999878
Epoch 5 batch 20 total loss: 1.379643201828003
Epoch 6 batch 0 total loss: 1.0654596090316772
Epoch 6 batch 10 total loss: 1.248755693435669
Epoch 6 batch 20 total loss: 1.4343621730804443
Epo

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# evaluate the model
Siamese.eval()

# Embed all labels with the trained network 
intents_embeddings = torch.zeros((n_classes, Siamese.output_size)).cuda()
intent_set = list(set(train_labels))
for i in range(len(intent_set)):
  lab = intent_set[i]
  intents_embeddings[i,:] = Siamese.FF(intent_centroids[lab].cuda())

In [ ]:
# Say we have a new query for which we want to find closeby centroids
i = 205
query = test_queries[i]
print("Test Query: ",query)
print("Test Label: ",label_map[test_labels[i]])

# Embed the query
encoded_query = tokenizer(query, padding=True, truncation=True,return_token_type_ids=False, max_length=128, return_tensors='pt')
encoded_query.to(device)
query_embeddings = Siamese.forward_one(encoded_query)

Test Query:  Could you tell me what your exchange rates are?
Test Label:  exchange rate


In [ ]:
# Test in a loop 
np.random.seed(42)
test_ix = np.random.choice(len(test_queries), 500)
top_k = 1 # the number of items predict
n_correct_SNN = 0
n_correct_CentNN = 0
n_correct_SBERT = 0

for ix in test_ix:
  query = test_queries[ix]
  label = label_map[test_labels[ix]]

  # LM + SNN's FF layer 
  encoded_query = tokenizer(query, padding=True, truncation=True,return_token_type_ids=False, max_length=128, return_tensors='pt')
  encoded_query.to(device)
  query_embeddings_SNN = Siamese.forward_one(encoded_query)
  top_intents = torch.topk(torch.norm(query_embeddings_SNN-intents_embeddings, 2, dim = 1), k = top_k, largest = False).indices.squeeze()
  if top_k == 1:
    top_intents = [top_intents]
  top_intents_set = []
  for i in range(len(top_intents)):
    label_ix = intent_set[top_intents[i].item()]
    top_intents_set.append(label_map[label_ix])
  if label in top_intents_set:
    n_correct_SNN+=1

  # With unsupervised SBERT embeddings: get the embedding label corresponding to the largest dot product 
  lm_enc = lm(**encoded_query) # ONLY use the language model
  query_embeddings_LM = mean_pooling(lm_enc, encoded_query['attention_mask']) 


  top_intents = torch.topk(torch.inner(query_embeddings_LM, intents_embeddings_raw), k = top_k).indices.squeeze()
  if top_k == 1:
    top_intents = [top_intents]
  top_intents_set = []
  for i in range(len(top_intents)):
    label_ix = intent_set[top_intents[i].item()]
    top_intents_set.append(label_map[label_ix])
  if label in top_intents_set:
    n_correct_SBERT+=1

  # With language model (LM) centroids (before SNN's FF layer) 
  top_intents_set = []
  top_intents = torch.topk(torch.inner(query_embeddings_LM, intent_centroids_lm), k = top_k).indices.squeeze()
  if top_k == 1:
    top_intents = [top_intents]
  for i in range(len(top_intents)):
    label_ix = intent_set[top_intents[i].item()]
    top_intents_set.append(label_map[label_ix])
  if label in top_intents_set:
    n_correct_CentNN+=1

In [ ]:
print("Unsupervised Methods")
print("SBERT: {}".format(n_correct_SBERT/len(test_ix)))

print("Semisupervised Methods")
print("CentNN: {}".format(n_correct_CentNN/len(test_ix)))
print("SNN: {}".format(n_correct_SNN/len(test_ix)))

Unsupervised Methods
SBERT: 0.466
Semisupervised Methods
CentNN: 0.506
SNN: 0.534


Could you tell me what your exchange rates are?
30 labels
semisupervised data has 20 examples of each label 
1. sample 5 queries of each label to use in batch --> 150 samples  
2. loop through each query: 
  for each query:
  - find hard positive (1 of 4)
  - find hard negative (1 of 145) 
  For centroids
  - find [pos centroid] repsective centroid embedding for Q 
  - find [neg centroid] random centroid embedding for Q NOT the same class

2 triplets per query --> 300 triplets 

In [ ]:
Could you tell me what your exchange rates are?